1- **Twitter EXERCISE:** 

Given the starting user "eloipuertas", retrieve the user name, the date of the accoount creation and the last tweet date and text for each follower. Store them to a database called 'twitter' and into a collection called 'info'. The attributes of the document are: 'screen_name', 'created_at', 'last_tweet_text' and 'last_tweet_created_at'. 


In [1]:
import time
import pymongo
import requests
import tweepy
from selenium.webdriver.common.keys import Keys
from selenium import webdriver
from selenium.webdriver.support.ui import Select

In [1]:
def retrieve(user_name):
    
    def defineBBDD():
        #Let's open our MongoDB credentials and read them.
        #PLEASE CREATE A 'credentials.txt' file in the same folder as this Notebook and write on separate but contiguous
        #lines the name, password and url to be used, like:
        
        #dbUser
        #password
        #somethingsomething.olevp.azure.mongodb.net/test
        
        try:
            with open("credentials.txt", 'r', encoding='utf-8') as f:
                [name,password,url]=f.read().splitlines()
                conn=pymongo.MongoClient("mongodb+srv://{}:{}@{}".format(name,password,url))
            print ("Connected successfully!!!")
        except pymongo.errors.ConnectionFailure as e:
            print ("Could not connect to MongoDB: %s" % e) 
        
        # create db and collection with the indicated names
        db = conn["Twitter"]
        return db['info']
    
    def setAuthAPI():
        #setup API:
        #PLEASE CREATE A 'APIKeys.txt' file in the same folder as this Notebook and write on separate but contiguous
        #lines your API keys in this order:
        
        
        #<CLIENT_KEY>
        #<CLIENT_SECRET>
        #<RESOURCE_OWNER_KEY>
        #<RESOURCE_OWNER_SECRET>
        
        with open("APIKeys.txt", 'r', encoding='utf-8') as f:
                [CLIENT_KEY,CLIENT_SECRET,RESOURCE_OWNER_KEY,RESOURCE_OWNER_SECRET]=f.read().splitlines()
        
        auth = tweepy.OAuthHandler(CLIENT_KEY, CLIENT_SECRET)
        auth.set_access_token(RESOURCE_OWNER_KEY, RESOURCE_OWNER_SECRET)
        
        return tweepy.API(auth,wait_on_rate_limit=True)
    
    def get_followers(api,user_name):
        users = []
        #Getting all the followers and storing them on an array for later use
        for i, user in enumerate(tweepy.Cursor(api.followers_ids, id=user_name).pages()):
            print('Getting followers')
            users += user
        print("Finished")
        return users
    
    def insert(api, collection, user_name):
        #Compute execution time
        start_time = time.time()
        #Retrieve followers
        users = get_followers(api,user_name)
        for i in users:
            #print("Trying user ",i)
            #Get user timeline WITHOUT RT's, aka get the last tweet (.items(1))
            tweets = tweepy.Cursor(api.user_timeline, user_id=i,tweet_mode='extended',include_rts=False).items(1)
            #This try is because there's people with private accounts and we cannot get their tweets.
            try:
                #This for is just for iteration
                for j in tweets:
                    text = j.full_text
                    date = j.created_at
                #Store on DDBB
                collection.insert_one({'screen_name':j.user.screen_name, 'created_at':j.user.created_at,
                                      'last_tweet_text':text,
                                       'last_tweet_created_at':date})
                #print("User ",j.user.screen_name," inserted")
            except tweepy.TweepError as e:
                pass
                #print("User ",i," is private or no longer exists.")

        print("All users inserted in ",(time.time()-start_time)/60.0," minutes. Check them out: https://cloud.mongodb.com/ \n")
        #PRINTING
        getAll = collection.find({})
        for document in getAll: 
            print(document['created_at'],document['screen_name'],"\n",
                  document['last_tweet_created_at'],document["last_tweet_text"]+"\n")

    insert(setAuthAPI(),defineBBDD(),user_name)

In [5]:
retrieve('emedemanel') #A user with few followers

Connected successfully!!!
Getting followers
Finished
All users inserted in  0.17202953497568765  minutes. Check them out: https://cloud.mongodb.com/ 

2020-08-04 07:42:41 GlendaRedondo 
 2020-10-02 09:57:57 @eganbernalismo vaya tela todo solo han querido sacarnos la pasta a todos en la extra xd

2019-10-14 13:21:44 ginaboix 
 2020-10-19 22:41:48 @maariadess 💜

2010-07-29 09:10:56 binary_spain 
 2020-10-20 14:23:00 fin

2012-10-04 16:54:10 Miguelrcde 
 2019-09-24 18:38:35 Participa en el concurs de #JoTMBé i guanya una invitació doble per gaudir de les pròximes edicions de @Brunch_BCN al llarg de setembre. https://t.co/LZcapK2Qk8

2012-09-27 19:13:48 MarkOrozco_97 
 2020-10-13 20:58:18 Completamente de acuerdo con la reflexión de Casillas entre Messi y CR7. Talento vs. Trabajo. https://t.co/puHGvY1cDZ

2020-07-01 21:22:57 yungbeefbot 
 2020-10-20 14:35:37 Que le follen a los avisos de desahucio

2012-02-10 22:28:04 uriburi3 
 2018-10-23 16:20:03 @pauvicentem @FCBarcelonaB Pau, un honor 

2- **Selenium EXERCISE:** 
<ul>
<li> Open a browser 

<li>Go to tripadvisor/Restaurants

<li>Find the search text box

<li>Clear it, input the query "Sant Cugat" and send it

<li>Go to "Restarurants" and get the name and link of the top 10 restaurants in Sant cugat
<li> Store them into a database called 'TripAdvisor' and a collection called 'Restaurants' with the documents atributes 'name' and 'link'
</ul>

</div>

In [6]:
def getRestaurants(location):
    
    def defineDDBB():
        #Let's open our MongoDB credentials and read them.
        #PLEASE CREATE A 'credentials.txt' file in the same folder as this Notebook and write on separate but contiguous
        #lines the name, password and url to be used, like:
        
        #dbUser
        #password
        #somethingsomething.olevp.azure.mongodb.net/test
        try:
            with open("credentials.txt", 'r', encoding='utf-8') as f:
                [name,password,url]=f.read().splitlines()

                conn=pymongo.MongoClient("mongodb+srv://{}:{}@{}".format(name,password,url))
            print ("Connected successfully!!!")
        except pymongo.errors.ConnectionFailure as e:
            print ("Could not connect to MongoDB: %s" % e) 
        # create db and collection
        db = conn["TripAdvisor"]
        return db['Restaurants']
        
        
    def scrap(collection):
        #Define the chromedriver and the url we want to go
        browser = webdriver.Chrome("chromedriver")
        browser.get("https://www.tripadvisor.es/Restaurants")

        #Find the search bar
        search_bar = browser.find_element_by_css_selector("input[placeholder='Ciudad o nombre del restaurante']")
        search_bar.clear()
        #Input the query
        search_bar.send_keys("Sant Cugat")
        #Those sleeps are depending on the machine performance on each one.
        time.sleep(2)
        #Select the first location suggested
        search_bar.send_keys(Keys.DOWN)
        time.sleep(2)
        #Search!
        search_bar.send_keys(Keys.ENTER)
        
        #Get the div with the query result
        restaurants = browser.find_elements_by_xpath('//div[@data-test-target="restaurants-list"]')
        
        #They are all named "X_list_item" where X is a number. We are only interested on the 10 first.
        #So i created a list with the div name for each restaurant
        restNumber = [str(i+1)+"_list_item" for i in range(10)]
        
        #Iterate over the 10 restaurants
        for i in restNumber:
            #Getting the div where the link and name are stored
            div = browser.find_element_by_xpath('//div[@data-test="%s"]' % i)
            restName = div.text.split("\n")[0]
            link = div.find_element_by_tag_name("a")
            collection.insert_one({'name':restName,'link':"https://www.tripadvisor.es/"+link.get_attribute("href")})

        print("Scrapped! Check the result in https://cloud.mongodb.com/")
        
        #PRINTING
        getAll = collection.find({})
        for document in getAll: 
            print(document['name'],"\n",document['link'])

    scrap(defineDDBB())

In [7]:
getRestaurants("Sant Cugat")

Connected successfully!!!
Scrapped! Check the result in https://cloud.mongodb.com/
1. Piaceri D’ Italia Ristorante Pizzeria 
 https://www.tripadvisor.es/https://www.tripadvisor.es/Restaurant_Review-g1080422-d19084243-Reviews-Piaceri_D_Italia_Ristorante_Pizzeria-Sant_Cugat_del_Valles_Catalonia.html
2. Restaurant Brau 
 https://www.tripadvisor.es/https://www.tripadvisor.es/Restaurant_Review-g1080422-d10195584-Reviews-Restaurant_Brau-Sant_Cugat_del_Valles_Catalonia.html
3. La Rita 
 https://www.tripadvisor.es/https://www.tripadvisor.es/Restaurant_Review-g1080422-d10365477-Reviews-La_Rita-Sant_Cugat_del_Valles_Catalonia.html
4. Sabàtic 
 https://www.tripadvisor.es/https://www.tripadvisor.es/Restaurant_Review-g1080422-d10167691-Reviews-Sabatic-Sant_Cugat_del_Valles_Catalonia.html
5. Némesis Gastronomia 
 https://www.tripadvisor.es/https://www.tripadvisor.es/Restaurant_Review-g1080422-d11892809-Reviews-Nemesis_Gastronomia-Sant_Cugat_del_Valles_Catalonia.html
6. El vi de deu 
 https://www.tri